In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from models.arima_forecasts import ARIMA_forecast, find_arima_spec
from models.random_walk import random_walk_forecast
from models.ann import ANN_forecast
from helper_functions import compute_rmse, load_monthly_baci_data, load_daily_baci_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning


# Suppress all warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ValueWarning)

In [ ]:
from models.ann import ANN_diff_forecast


def ARIMA_vs_RW_vs_ANN(data, train_ratio=0.8, steps_ahead=1, n_forecasts=1, plot=False, res=15, arima_spec=(1, 1, 2), ann_spec=(2, 2), plot_all=True):

    def plot_forecast(arima_fitted_df, 
                      arima_forecast_df, 
                      rw_forecast_df,
                      ann_fitted_df,
                      ann_forecast_df,
                      train_df,
                      test_df,
                      res=15):
        true_values = pd.concat([train, test])

        # Plot
        plt.figure(figsize=(15,7))

        plt.plot(true_values.index[-res - steps_ahead:], true_values[-res - steps_ahead:].values, color="blue", label='observed')
        plt.plot(arima_fitted_df.index[-res:], arima_fitted_df[-res:].values, color='green', label='ARIMA fitted')
        plt.plot(ann_fitted_df.index[-res:], ann_fitted_df[-res:].values, color='red', label='ANN fitted')
        
        
        # Rename columns of train_df.iloc[-1] to match rw_forecast_df's columns
        # Convert the last value of train_df to a dataframe with the same column name as rw_forecast_df
        renamed_row = pd.DataFrame([arima_fitted_df.values[-1]], columns=arima_forecast_df.columns, index=[arima_fitted_df.index[-1]])
        arima_forecast_df_merged = pd.concat([renamed_row, arima_forecast_df])
    
        renamed_row = pd.DataFrame([ann_fitted_df.values[-1]], columns=ann_forecast_df.columns, index=[ann_fitted_df.index[-1]])
        ann_forecast_df_merged = pd.concat([renamed_row, ann_forecast_df])
      
        renamed_row = pd.DataFrame([train_df.iloc[-1]], columns=rw_forecast_df.columns, index=[train_df.index[-1]])
        rw_forecast_df_merged = pd.concat([renamed_row, rw_forecast_df])


        plt.plot(arima_forecast_df_merged.index, arima_forecast_df_merged.values, color='green', label='ARIMA forecast', linestyle=":")
        plt.plot(rw_forecast_df_merged.index, rw_forecast_df_merged.values, color="purple", label="RW forecast", linestyle=":")
        plt.plot(ann_forecast_df_merged.index, ann_forecast_df_merged.values, color="red", label="ANN forecast", linestyle=":")

        plt.legend()
        plt.show()
        print(f"ARIMA MSE: {compute_rmse(test, arima_forecast_df)}")
        print(f"RW MSE: {compute_rmse(test, rw_forecast_df)}")
        print(f"ANN MSE: {compute_rmse(test, ann_forecast_df)}")




    train_size = int(train_ratio * len(data))
    arima_rmses, rw_rmses, ann_rmses = [], [], []
    
    # Initialize empty DataFrames with 'Date' as the index
    columns = ['Forecast']
    first_train_size = train_size
    arima_all_forecasts_df = rw_all_forecasts_df = ann_all_forecasts_df = pd.DataFrame(columns=columns).set_index(pd.DatetimeIndex([], name='Date'))
    
    for _ in range(n_forecasts):
        print("Forecast number " + str(_ + 1))
        train = data["CLOSE"].iloc[:train_size]
        test = data["CLOSE"].iloc[train_size:train_size + steps_ahead]

        # ARIMA forecast
        arima_fitted_df, arima_forecast_df, arima_model = ARIMA_forecast(train, 
                                                                         test, 
                                                                         steps_ahead, 
                                                                         p=arima_spec[0], 
                                                                         i=arima_spec[1], 
                                                                         q=arima_spec[2],
                                                                         plot=False)
        #print(arima_forecast_df.values)
        arima_rmse = compute_rmse(test, arima_forecast_df)
        arima_rmses.append(arima_rmse)

        # Random Walk forecast
        rw_forecast_df = random_walk_forecast(train, test)
        rw_rmse = compute_rmse(test, rw_forecast_df)
        rw_rmses.append(rw_rmse)

        # ANN forecast
        ann_fitted_df, ann_forecast_df = ANN_diff_forecast(train=train, 
                                                      test=test, 
                                                      steps_ahead=steps_ahead,
                                                      lookback=ann_spec[0],
                                                      hidden_units=ann_spec[1])
        ann_rmse = compute_rmse(test, ann_forecast_df)
        ann_rmses.append(ann_rmse)
        
        # Add to all forecasts
        arima_all_forecasts_df = pd.concat([arima_all_forecasts_df, arima_forecast_df])
        ann_all_forecasts_df = pd.concat([ann_all_forecasts_df, ann_forecast_df])
        rw_all_forecasts_df = pd.concat([rw_all_forecasts_df, rw_forecast_df])


        train_size += steps_ahead

        if plot:
            plot_forecast(arima_fitted_df=arima_fitted_df,
                          arima_forecast_df=arima_forecast_df,
                          rw_forecast_df=rw_forecast_df,
                          ann_fitted_df=ann_fitted_df,
                          ann_forecast_df=ann_forecast_df,
                          train_df=train,
                          test_df=test,
                          res=res)
    if plot_all:
        plt.gca().set_prop_cycle(None)  # Reset color cycle
        plt.plot(data.index[first_train_size:train_size + steps_ahead - 1], data[first_train_size:train_size + steps_ahead - 1].values, color="blue", label='observed')
        plt.gca().set_prop_cycle(None)  # Reset color cycle
        plt.plot(arima_all_forecasts_df.index, arima_all_forecasts_df.values, color='green', label='ARIMA forecast', linestyle=":")
        plt.plot(rw_all_forecasts_df.index, rw_all_forecasts_df.values, color='purple', label='RW forecast', linestyle=":")
        plt.plot(ann_all_forecasts_df.index, ann_all_forecasts_df.values, color='red', label='ANN forecast', linestyle=":")

        # Rotate x-axis labels for better clarity
        plt.xticks(rotation=45)

        # Optional: Use fewer date ticks on the x-axis
        #import matplotlib.dates as mdates
        #plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))  # Every week

        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())  # Display unique labels only

        plt.tight_layout()  # Adjust the layout for better display
        plt.show()

        


    return arima_rmses, rw_rmses, ann_rmses






In [ ]:
df = load_monthly_baci_data()
#find_arima_spec(df)

In [ ]:
train_ratio = 0.7
n_forecasts = 50
steps_ahead = 1
res = 200
arima_spec = (1, 1, 1)
ann_spec = (1, 2)
plot = False
plot_all = True
arima_rmses, rw_rmses, ann_rmses = ARIMA_vs_RW_vs_ANN(df, 
                                                      train_ratio=train_ratio, 
                                                      n_forecasts=n_forecasts, 
                                                      steps_ahead=1, 
                                                      plot=False, 
                                                      res=200, 
                                                      arima_spec=(1, 1, 2), 
                                                      ann_spec=(1, 2),
                                                      plot_all=True)

print(f"ARIMA RMSEs: {np.average(arima_rmses)}")
print(f"Random Walk RMSEs: {np.average(rw_rmses)}")
print(f"ANN RMSEs: {np.average(ann_rmses)}")
# Open the file in write mode
with open('log.txt', 'a') as file:
    print("Monthly Forecast: Train ratio: " + str(train_ratio) + ". " + str(steps_ahead) + " step ahead. " + str(n_forecasts) + " forecasts", file=file)
    print(f"ARIMA RMSEs: {np.average(arima_rmses)}", file=file)
    print(f"Random Walk RMSEs: {np.average(rw_rmses)}", file=file)
    print(f"ANN RMSEs: {np.average(ann_rmses)}", file=file)

In [ ]:
df = load_daily_baci_data()
#find_arima_spec(df)

In [ ]:
train_ratio = 0.5
n_forecasts = 50
steps_ahead = 1
res = 200
arima_spec = (1, 1, 2)
ann_spec = (1, 2)
plot = False
plot_all = True
arima_rmses, rw_rmses, ann_rmses = ARIMA_vs_RW_vs_ANN(df, 
                                                      train_ratio=train_ratio, 
                                                      n_forecasts=n_forecasts, 
                                                      steps_ahead=1, 
                                                      plot=False, 
                                                      res=200, 
                                                      arima_spec=(1, 1, 2), 
                                                      ann_spec=(1, 2),
                                                      plot_all=True)

print(f"ARIMA RMSEs: {np.average(arima_rmses)}")
print(f"Random Walk RMSEs: {np.average(rw_rmses)}")
print(f"ANN RMSEs: {np.average(ann_rmses)}")
# Open the file in write mode
with open('log.txt', 'a') as file:
    print("Daily Forecast: Train ratio: " + str(train_ratio) + ". " + str(steps_ahead) + " step ahead. " + str(n_forecasts) + " forecasts", file=file)
    print(f"ARIMA RMSEs: {np.average(arima_rmses)}", file=file)
    print(f"Random Walk RMSEs: {np.average(rw_rmses)}", file=file)
    print(f"ANN RMSEs: {np.average(ann_rmses)}", file=file)
    